In [ ]:
import numpy as np
import pandas as pd
from tqdm import tqdm

In [ ]:
def convert_clock_to_mins(time_str):
    minutes, seconds = map(int, time_str.split(":"))
    return minutes

In [ ]:
def mins_in_range(mins, lower_bound):
  return int(mins) >= lower_bound and int(mins) < lower_bound + 1

In [ ]:
def get_pts_min(i, all_shots):
  df = all_shots[all_shots["Made"] == True]
  pts_sum = 0
  for index, row in df.iterrows():
    if row["GameClock"] >= i and row["GameClock"] < i + 1:
      pts_sum += row["Value"]

  return pts_sum / len(np.unique(all_shots["Player"]))

In [ ]:
def add_game_clock(all_shots):
  GameClock = []
  for index, row in all_shots.iterrows():
      GameClock.append(12 * (row["Period"] - 1) + (11 - convert_clock_to_mins(row["ClockTime"])))
  all_shots["GameClock"] = GameClock

  return all_shots

In [ ]:
def get_all_pts_per_min(all_shots):
  pts = []
  for i in tqdm(range(48)):
    pts.append(get_pts_min(i, all_shots))

  return pd.DataFrame({"MIN": range(48), "PTS": pts})

In [ ]:
all_shots = pd.read_csv("pbpstats-tracking-shots.csv", engine="c")

In [ ]:
all_shots = add_game_clock(all_shots)

In [ ]:
ppm_data = get_all_pts_per_min(all_shots)

100%|██████████| 48/48 [00:16<00:00,  2.84it/s]


In [ ]:
ppm_data.to_csv("ppm_data.csv")

Construct L

In [ ]:
def L(n, k, ppm_data, use_scaled=True):

  if use_scaled:
    pts_label = "SCALED_PTS"
  else:
    pts_label = "PTS"
  p = .08
  time_until_foul_out = int(np.floor((6 - k) / p))
  time_remaining_in_game_after_reinsertion = 48 - 8*k
  sum = 0

  if time_until_foul_out <= time_remaining_in_game_after_reinsertion:
    print("Using time until foul out")
    for j in range(time_until_foul_out + 1):
      if ((n+j) < 48 and (8*k+j) < 48):
        phi_i = ppm_data[ppm_data["MIN"] == (n+j)][pts_label].values[0]
        phi_f = ppm_data[ppm_data["MIN"] == (8*k+j)][pts_label].values[0]
        sum += (phi_i - phi_f)

    return sum
  elif time_until_foul_out > time_remaining_in_game_after_reinsertion:
    print("Using time remaining in game")
    for j in range(time_until_foul_out + 1):
      if ((n+j) < 48):
        phi_i = ppm_data[ppm_data["MIN"] == (n+j)][pts_label].values[0]
        sum += phi_i

    for j in range(time_remaining_in_game_after_reinsertion + 1):
      if ((8*k+j) < 48):
        phi_f = ppm_data[ppm_data["MIN"] == (8*k+j)][pts_label].values[0]
        sum -= phi_f

    return sum


In [ ]:
def scale_mins(ppm_data, q2_const = 2.17, q3_const = 3.55, q4_const = 21.08):
  scaled_pts = []
  for index, row in ppm_data.iterrows():
    if 0 <= row["MIN"] < 12:
      scaled_pts.append(row["PTS"])
    elif 12 <= row["MIN"] < 24:
      scaled_pts.append(q2_const * row["PTS"] )
    elif 24 <= row["MIN"] < 36:
      scaled_pts.append(q3_const * row["PTS"] )
    elif 36 <= row["MIN"] < 48:
      scaled_pts.append(q4_const * row["PTS"] )
    else:
      scaled_pts.append(q4_const * row["PTS"])

  return scaled_pts

In [ ]:
ppm_data[ppm_data["MIN"] == 47]

,MIN,PTS
47,47,1.103837


In [ ]:
ppm_data["SCALED_PTS"] = scale_mins(ppm_data)

In [ ]:
L(45, 0, ppm_data, use_scaled=True)

Using time remaining in game


-356.00282167042883